# "COVID-19 - Overview"
> Many countries in one large overview. Updated every day.

- comments: true
- author: Caglar Cakan
- categories: [covid19]
- permalink: /covid-dashboard-v2/
- image: images/icon_covid_dashboard_2.png
- license: Creative Commons

In [1]:
#hide_input
import datetime
print("This page was last built on", datetime.datetime.now().strftime("%d.%m.%y %H:%M:%S"))

This page was last built on 07.12.20 03:56:06


**Colors:** 
If the last reported cases are < 25% of the maximum, the data is shown in green. Else, if the last cases are > 75% of the maximum, the data is shown in red. Else, grey.

**Axes:** 
The data is shown on a **normalized y-axis**. This means that the actual numbers between countries can be very different and should not be compared. On the x-axis, the data is shown from the day of the 100th positive case until today.

**Preprocessing:**
The number of new cases per day is averaged using a 10-day rolling window. Countries are included if they have at least 1000 cases and at least 100 deaths. Countries are sorted by total number of cases. 

**Data:** The data is pulled from the COVID-19 [Data Repository](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series) by Johns Hopkins CSSE

In [2]:
#hide
import numpy as np

import datetime

import matplotlib.pyplot as plt
import pandas as pd

import requests
import io
import math
import textwrap

from countryinfo import CountryInfo

In [3]:
#hide
import lib.covid_data as cd

In [4]:
#hide
import altair as alt
import numpy as np
import pandas as pd

BG_COLOR = '#282C34'
FG_COLOR = '#7d828c'
RED_COLOR = '#FF0000'
GREEN_COLOR = '#00FF7F'
WHITE_COLOR = '#ABB2BF'
MUTED_FG_COLOR = '#5d6169'

PLOTS_PER_ROW = 3
ROLLING_MEAN_DAYS = 10

PLOT_WIDTH = 150
PLOT_HEIGHT = 150
PLOT_STROKE = 3

def altair_plot_country(str_country, xlabel=True, ylabel=True):
    # get data
    df_cases_country = dff2[dff2['country']==str_country].copy()
    rolling_mean = df_cases_country.new_confirmed.rolling(ROLLING_MEAN_DAYS, min_periods=1).mean()
    df_cases_country['rolling_mean'] = rolling_mean
    
    # determine color of country
    last_ratio = rolling_mean.iloc[-1] / rolling_mean.max() # last value compared to maximum
    plot_color = RED_COLOR if last_ratio > 0.75 else GREEN_COLOR if last_ratio < 0.25 else WHITE_COLOR

    axis_properties = axis=alt.Axis(
        labelColor=FG_COLOR, 
        titleColor=FG_COLOR,
        domainWidth=0,
    )

    country_chart = alt.Chart(df_cases_country).mark_line(strokeWidth=PLOT_STROKE).encode(
        alt.X('date:T', title="Time" if xlabel else None, axis=axis_properties),
        alt.Y('rolling_mean:Q', title="Daily cases" if ylabel else None, axis=axis_properties),
        color=alt.value(plot_color),
    ).properties(
        width=PLOT_WIDTH,
        height=PLOT_HEIGHT,
        title=str_country,
    )
    return country_chart

def build_combined(countries):
    ncols = len(countries) if len(countries) <= PLOTS_PER_ROW else PLOTS_PER_ROW
    nrows = math.ceil(len(countries)/ncols)
    i = -1
    col_plots = None # holds vertically stacked plots
    for r in range(nrows):
        row_plots = None # holds horizontally stacked plots
        for c in range(ncols):
            i+=1
            # if there are no countries left to plot
            if i==len(countries):
                break
            str_country = countries[i]
            country_plot = _altair_plot_country(str_country, ylabel=True if c == 0 else False, xlabel=True if r == nrows-1 else False)
            # altair horizontal stacking
            row_plots = country_plot if row_plots is None else row_plots | country_plot
        # altair vertical stacking
        col_plots = row_plots if col_plots is None else col_plots & row_plots
    return col_plots

def make_plot(countries):
    combined = build_combined(countries)

    combined = combined.configure(
        #background=BG_COLOR
    ).configure_axis(
        grid=False,
    ).configure_view(
        strokeWidth=0,
    ).configure_title(
        fontSize=18,
        #font='Courier',
        anchor='start',
        color=FG_COLOR
    )
    return combined

In [5]:
#hide
def _altair_plot_country(str_country, xlabel=True, ylabel=True):
    # get data
    df_cases_country = dff2[dff2['country']==str_country].copy()
    rolling_mean = df_cases_country.new_confirmed.rolling(ROLLING_MEAN_DAYS, min_periods=1).mean()
    df_cases_country['rolling_mean'] = rolling_mean
    
    # determine color of country
    last_ratio = rolling_mean.iloc[-1] / rolling_mean.max() # last value compared to maximum
    plot_color = RED_COLOR if last_ratio > 0.75 else GREEN_COLOR if last_ratio < 0.25 else MUTED_FG_COLOR

    axis_properties = axis=alt.Axis(
        labelColor=FG_COLOR, 
        titleColor=FG_COLOR,
        domainWidth=0,
    )
    
    # line charts
    base = alt.Chart(df_cases_country).encode(alt.X('date:T', title="Time" if xlabel else None, axis=axis_properties))

    noisey_chart = base.mark_bar(size=0.5).encode(
        alt.Y('new_confirmed:Q', title="New cases" if ylabel else None, axis=axis_properties),
        color=alt.value(WHITE_COLOR)
    )
    
    
    mean_chart = base.mark_line(strokeWidth=PLOT_STROKE).encode(
        alt.Y('rolling_mean:Q', title="New cases" if ylabel else None, axis=axis_properties),
        color=alt.value(plot_color)
    )
    
    
    # text annotation
    # -----------------------------
    # from https://altair-viz.github.io/gallery/multiline_tooltip.html
    # a selector
    nearest = alt.selection(
        type='single', nearest=True, on='mouseover',
        fields=['date'], empty='none')
    selectors = noisey_chart.mark_point().encode(
        alt.X('date:T'),
        opacity=alt.value(0),
    ).add_selection(
        nearest
    ) 
    
    # Draw points on the line, and highlight based on selection
    points = noisey_chart.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    text = points.mark_text(
        align='right', 
        dx=0, 
        dy=-10,
        color=WHITE_COLOR
    ).encode(
        text=alt.condition(nearest, 'new_confirmed:Q', alt.value(''))
    )       

    # combine everything
    country_chart = alt.layer(
        noisey_chart,
        mean_chart,
        selectors,
        points,
        text,
    )
    country_chart=country_chart.properties(
        width=PLOT_WIDTH,
        height=PLOT_HEIGHT,
        title=str_country,
    )

    return country_chart

In [6]:
#hide
MAX_COUNTRIES = 3 * 15

dff2 = cd.get_df(MIN_CASES = 500, SINCE_CASES = 100)
countries = cd.get_countries_with_min_cases(dff2, 'deaths', min_cases= 10)
countries = countries[:MAX_COUNTRIES]
#countries.sort()

In [7]:
#hide
countries

array(['US', 'Brazil', 'India', 'Mexico', 'United Kingdom', 'Italy',
       'France', 'Iran', 'Spain', 'Russia', 'Argentina', 'Colombia',
       'Peru', 'South Africa', 'Poland', 'Germany', 'Indonesia',
       'Belgium', 'Chile', 'Turkey', 'Ukraine', 'Ecuador', 'Canada',
       'Iraq', 'Romania', 'Netherlands', 'Bolivia', 'Czechia',
       'Philippines', 'Pakistan', 'Sweden', 'Bangladesh', 'Egypt',
       'Morocco', 'Saudi Arabia', 'Hungary', 'Switzerland', 'Portugal',
       'Bulgaria', 'China', 'Guatemala', 'Austria', 'Tunisia', 'Panama',
       'Jordan'], dtype=object)

In [8]:
#hide_input
make_plot(countries)

alt.VConcatChart(...)